# Module Installation

In [1]:
!pip install --upgrade scikit-image
!pip install --upgrade imutils
!pip install opencv
!pip install opencv-python

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\scikit_image-0.17.2-py3.8.egg-info\\dependency_links.txt'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.17.2
    Uninstalling scikit-image-0.17.2:
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25862 sha256=158757915976164abed7d47a538df646dae2102139ca5b6b9b41cda7f4f8d858
  Stored in directory: c:\users\elean\appdata\local\pip\cache\wheels\59\1b\52\0dea905f8278d5514dc4d0be5e251967f8681670cadd3dca89
Successfully built imutils


ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv


In [ ]:
!pip uninstall opencv
!pip uninstall opencv-python
!pip uninstall opencv-contrib-python

# Image Classification

## Capture Webcam

In [5]:
import datetime
import cv2
import os

cv2.namedWindow("Preview_Window")
vc = cv2.VideoCapture(0)

if vc.isOpened(): # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

while rval:
    cv2.imshow("Preview_Window", frame)    
    rval, frame = vc.read()
    
    key = cv2.waitKey(20)
    x = datetime.datetime.now()
    x = x.strftime("%Y%b%d %H%M%S")
    if key == 27: # exit on ESC
        # cv2.imwrite(os.path.join("./01_FrameOutput/", '%s_FinalFrame.png') % x, frame)
        # cv2.imwrite(os.path.join("./01_FrameOutput/", '%s_FinalFrame.png') % x, frame)
        cv2.imwrite('./01_FrameOutput/FinalFrame.png', frame)
        break        
    elif key == ord('s'): # wait for 's' key to save and exit
        # filename = 'Frame_' + x + '.png'
        # cv2.imwrite(filename, frame)
        # cv2.imwrite(os.path.join("./01_FrameOutput/", '%s_Frame.png') % x, frame)
        cv2.imwrite('./01_FrameOutput/FirstFrame.png', frame)
        
cv2.destroyWindow("Preview_Window")

## Viz Image Difference

In [5]:
from skimage.metrics import structural_similarity as compare_ssim
import cv2
import numpy as np

# before = cv2.imread('./01_FrameOutput/2021Jan26 091611_Frame.png')
# after = cv2.imread('./01_FrameOutput/2021Jan26 091605_Frame.png')
before = cv2.imread('./01_FrameOutput/FirstFrame.png')
after = cv2.imread('./01_FrameOutput/FinalFrame.png')

# Convert images to grayscale
before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)

# Compute SSIM between two images
(score, diff) = compare_ssim(before_gray, after_gray, full=True)
print("Image similarity", score)

# The diff image contains the actual image differences between the two images
# and is represented as a floating point data type in the range [0,1] 
# so we must convert the array to 8-bit unsigned integers in the range
# [0,255] before we can use it with OpenCV
diff = (diff * 255).astype("uint8")

# Threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

mask = np.zeros(before.shape, dtype='uint8')
filled_after = after.copy()

for c in contours:
    area = cv2.contourArea(c)
    if area > 40:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(before, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.rectangle(after, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.drawContours(mask, [c], 0, (0,255,0), -1)
        cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)

# cv2.imshow('before', before)
cv2.imshow('after', after)
# cv2.imshow('diff',diff)
# cv2.imshow('mask',mask)
# cv2.imshow('filled after',filled_after)
cv2.waitKey(0)

Image similarity 0.867325467356008


-1

# Object Detection YOLO

## Model Buildup

In [2]:
import numpy as np
import datetime
import cv2
import os

# Load Yolo
print("LOADING YOLO")
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
# save all the names in file o the list classes
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
    
# get layers of the network
layer_names = net.getLayerNames()
# Determine the output layer names from the YOLO model
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print("YOLO LOADED")

LOADING YOLO


error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:207: error: (-212:Parsing error) Failed to parse NetParameter file: yolov3.cfg in function 'cv::dnn::dnn4_v20211004::readNetFromDarknet'


## Object Webcam Detection

In [1]:
video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    re, img = video_capture.read()
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # USing blob function of opencv to preprocess image
    blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),
                                 swapRB=True, crop=False)
    # Detecting objects
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # We use NMS function in opencv to perform Non-maximum Suppression
    # we give it score threshold and nms threshold as arguments.
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 2, color, 3)

    cv2.imshow("Image", cv2.resize(img, (800, 600)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

## Train YOLO Model To Detect Custome Object

# Web Server Deploument

In [3]:
!pip install streamlit
!pip install --upgrade streamlit opencv-python

Requirement already up-to-date: streamlit in c:\programdata\anaconda3\lib\site-packages (0.75.0)
Requirement already up-to-date: opencv-python in c:\programdata\anaconda3\lib\site-packages (4.5.1.48)


In [6]:
!streamlit run https://raw.githubusercontent.com/streamlit/demo-self-driving/master/streamlit_app.py

^C


# OCV

In [9]:
!pip install opencv-python
!pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.8-py2.py3-none-any.whl size=14067 sha256=c911bd3099e13c220d2890248f2d91ef93216f89a406166b37c76012ea632bd0
  Stored in directory: c:\users\2945483\appdata\local\pip\cache\wheels\ed\8a\c6\40c7ec06c2dd3df636832537238128a1471bf9c1b6a3a9bf40
Successfully built pytesseract


In [5]:
# Import required packages
import cv2
import pytesseract
 
# Mention the installed location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\2945483\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
# r'C:\Users\USER\AppData\Local\Tesseract-OCR\tesseract.exe'

# Read image from which text needs to be extracted
img = cv2.imread("Calix_Label_001.jpg")

# Preprocessing the image starts
 
# Convert the image to gray scale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite('./01_FrameOutput/Gray.png', gray)

# Performing OTSU threshold
# ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# Specify structure shape and kernel size.
# Kernel size increases or decreases the area
# of the rectangle to be detected.
# A smaller value like (10, 10) will detect
# each word instead of a sentence.
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
 
# Applying dilation on the threshold image
dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)
 
# Finding contours
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
                                                 cv2.CHAIN_APPROX_NONE)
 
# Creating a copy of image
im2 = img.copy()

# A text file is created and flushed
file = open("recognized.txt", "w+")
file.write("")
file.close()
 
# Looping through the identified contours
# Then rectangular part is cropped and passed on
# to pytesseract for extracting text from it
# Extracted text is then written into the text file
custom_config = r'--oem 3 --psm 6'
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
     
    # Drawing a rectangle on copied image
    rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
     
    # Cropping the text block for giving input to OCR
    cropped = im2[y:y + h, x:x + w]
    # cv2.imshow('threshold image', cropped)
    
    # Open the file in append mode
    file = open("recognized.txt", "a")
     
    # Apply OCR on the cropped image
    text = pytesseract.image_to_string(cropped, config=custom_config, lang='eng')
    print(text)
     
    # Appending the text into file
    file.write(text)
    file.write("\n")
     
    # Close the file
    file.close
    
cv2.imshow('threshold image', thresh1)

cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [46]:
# Import required packages
import cv2
import pytesseract
import easyocr

from pytesseract import Output
 
# Mention the installed location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# r'C:\Users\USER\AppData\Local\Tesseract-OCR\tesseract.exe'

# Read image from which text needs to be extracted
img = cv2.imread("D:\\24_R&D_Projects\\AIVisionOCR\\AIVisionOCR\\CalixLabel.jpg")
# img = cv2.imread("001.png")

gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
noise_remove_img = cv2.medianBlur(gray_image, 5)
blur = cv2.GaussianBlur(img,(5,5),0)
threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

h, w, c = img.shape
boxes = pytesseract.image_to_boxes(threshold_img) 
for b in boxes.splitlines():
    b = b.split(' ')
    x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
    cv2.rectangle(threshold_img, (x,y), (x+w,y+h), (50, 50, 255), 2)
    cv2.putText(threshold_img,b[0],(x,hImg- y+25),cv2.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)

cv2.imshow('noise_remove_img', threshold_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

custom_config = r'--psm 11'
text = pytesseract.image_to_string(threshold_img, config=custom_config)
print(text)

# A text file is created and flushed
file = open("recognized.txt", "w+")
file.write(text)
file.write("\n")
file.close()

ModuleNotFoundError: No module named 'easyocr'

In [35]:
custom_config = r'--oem 3 --psm 6'
details = pytesseract.image_to_data(threshold_img, output_type=Output.DICT, config=custom_config, lang='eng')
print(details.keys())

dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


In [26]:
total_boxes = len(details['text'])
for sequence_number in range(total_boxes):
    if int(float(details['conf'][sequence_number])) > 30:
        (x, y, w, h) = (details['left'][sequence_number], details['top'][sequence_number], details['width'][sequence_number],  details['height'][sequence_number])
        threshold_img = cv2.rectangle(threshold_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow('captured text', threshold_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Trial Run

In [28]:
!pip install pyocr

  Created wheel for pyocr: filename=pyocr-0.8-py3-none-any.whl size=36742 sha256=25dbd74a5728fb64b86b4933d76c9741e5cf870dd60a0b3e7ecf0e64d07f87c4
  Stored in directory: c:\users\2945483\appdata\local\pip\cache\wheels\3e\57\2b\21bf3422b0cb190028bd86d79b9daa9b61363fc73681956066
Successfully built pyocr


In [31]:
# imports:
import cv2
import numpy as np
# OCR imports:
from PIL import Image
import pyocr
import pyocr.builders

# image path
path = "D://opencvImages//"
fileName = "c552h.png"

# Reading an image in default mode:
inputImage = cv2.imread(path + fileName)
inputImage = cv2.imread("Calix_Label_001.jpg")

# Get local maximum:
kernelSize = 5
maxKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernelSize, kernelSize))
localMax = cv2.morphologyEx(inputImage, cv2.MORPH_CLOSE, maxKernel, None, None, 1, cv2.BORDER_REFLECT101)

# Perform gain division
gainDivision = np.where(localMax == 0, 0, (inputImage/localMax))

# Clip the values to [0,255]
gainDivision = np.clip((255 * gainDivision), 0, 255)

# Convert the mat type from float to uint8:
gainDivision = gainDivision.astype("uint8")

cv2.imshow('captured text', gainDivision)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import CalixOCR



In [2]:
python --version

NameError: name 'python' is not defined

In [3]:
!python --version

Python 3.8.5


In [37]:
import cv2
import pytesseract
import numpy as np
from PIL import ImageGrab
import time


pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
img = cv2.imread("D:\\24_R&D_Projects\\AIVisionOCR\\AIVisionOCR\\CalixLabel.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
pytesseract
##############################################
##### Image to String   ######
##############################################
# print(pytesseract.image_to_string(img))

#############################################
#### Detecting Characters  ######
#############################################
hImg, wImg,_ = img.shape
boxes = pytesseract.image_to_boxes(img)
for b in boxes.splitlines():
    print(b)
    b = b.split(' ')
    print(b)
    x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
    cv2.rectangle(img, (x,hImg- y), (w,hImg- h), (50, 50, 255), 2)
    cv2.putText(img,b[0],(x,hImg- y+25),cv2.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)


##############################################
##### Detecting Words  ######
##############################################
# #[   0          1           2           3           4          5         6       7       8        9        10       11 ]
# #['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text']
# boxes = pytesseract.image_to_data(img)
# for a,b in enumerate(boxes.splitlines()):
#         print(b)
#         if a!=0:
#             b = b.split()
#             if len(b)==12:
#                 x,y,w,h = int(b[6]),int(b[7]),int(b[8]),int(b[9])
#                 cv2.putText(img,b[11],(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)
#                 cv2.rectangle(img, (x,y), (x+w, y+h), (50, 50, 255), 2)


##############################################
##### Detecting ONLY Digits  ######
##############################################
# hImg, wImg,_ = img.shape
# conf = r'--oem 3 --psm 6 outputbase digits'
# boxes = pytesseract.image_to_boxes(img,config=conf)
# for b in boxes.splitlines():
#     print(b)
#     b = b.split(' ')
#     print(b)
#     x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
#     cv2.rectangle(img, (x,hImg- y), (w,hImg- h), (50, 50, 255), 2)
#     cv2.putText(img,b[0],(x,hImg- y+25),cv2.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)


##############################################
##### Webcam and Screen Capture Example ######
##############################################
# cap = cv2.VideoCapture(0)
# cap.set(3,640)
# cap.set(4,480)
# def captureScreen(bbox=(300,300,1500,1000)):
#     capScr = np.array(ImageGrab.grab(bbox))
#     capScr = cv2.cvtColor(capScr, cv2.COLOR_RGB2BGR)
#     return capScr
# while True:
#     timer = cv2.getTickCount()
#     _,img = cap.read()
#     #img = captureScreen()
#     #DETECTING CHARACTERES
#     hImg, wImg,_ = img.shape
#     boxes = pytesseract.image_to_boxes(img)
#     for b in boxes.splitlines():
#         #print(b)
#         b = b.split(' ')
#         #print(b)
#         x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
#         cv2.rectangle(img, (x,hImg- y), (w,hImg- h), (50, 50, 255), 2)
#         cv2.putText(img,b[0],(x,hImg- y+25),cv2.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)
#     fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
#     #cv2.putText(img, str(int(fps)), (75, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (20,230,20), 2);
#     cv2.imshow("Result",img)
#     cv2.waitKey(1)
#
#

cv2.imshow('img', img)
cv2.waitKey(0)

~ 0 0 1280 960 0
['~', '0', '0', '1280', '960', '0']


-1

In [40]:
import cv2
import pytesseract
import numpy as np
from PIL import ImageGrab
import time


pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
img = cv2.imread("D:\\24_R&D_Projects\\AIVisionOCR\\AIVisionOCR\\CalixLabel.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#############################################
#### Detecting Characters  ######
#############################################
hImg, wImg,_ = img.shape
boxes = pytesseract.image_to_boxes(img)
for b in boxes.splitlines():
    print(b)
    b = b.split(' ')
    print(b)
    x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
    cv2.rectangle(img, (x,y), (x+w,y+h), (50, 50, 255), 2)
    cv2.putText(img,b[0],(x,hImg- y+25),cv2.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)


cv2.imshow('img', img)
cv2.waitKey(0)

~ 0 0 1280 960 0
['~', '0', '0', '1280', '960', '0']


-1

In [48]:
!pip install easyocr

^C


In [ ]:
# Import required packages
import cv2
import pytesseract
import easyocr

from pytesseract import Output
 
# Mention the installed location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# r'C:\Users\USER\AppData\Local\Tesseract-OCR\tesseract.exe'

# Read image from which text needs to be extracted
img = cv2.imread("D:\\24_R&D_Projects\\AIVisionOCR\\AIVisionOCR\\CalixLabel.jpg")
# img = cv2.imread("001.png")

gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
noise_remove_img = cv2.medianBlur(gray_image, 5)
blur = cv2.GaussianBlur(img,(5,5),0)
threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

h, w, c = img.shape
boxes = pytesseract.image_to_boxes(threshold_img) 
for b in boxes.splitlines():
    b = b.split(' ')
    x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
    cv2.rectangle(threshold_img, (x,y), (x+w,y+h), (50, 50, 255), 2)
    cv2.putText(threshold_img,b[0],(x,hImg- y+25),cv2.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)

cv2.imshow('noise_remove_img', threshold_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

custom_config = r'--psm 11'
text = pytesseract.image_to_string(threshold_img, config=custom_config)
print(text)

# A text file is created and flushed
file = open("recognized.txt", "w+")
file.write(text)
file.write("\n")
file.close()

## OCR

In [2]:
# Import required packages
import cv2
import pytesseract

from pytesseract import Output
 
# Mention the installed location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# r'C:\Users\USER\AppData\Local\Tesseract-OCR\tesseract.exe'

# Read image from which text needs to be extracted
img = cv2.imread('C:/Users/elean/PythonProj/02_Medias/CalixLabel.jpg')
# img = cv2.imread("001.png")
# img = cv2.GaussianBlur(img, (5, 5), 0)

gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
otsu_threshold, threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
print(otsu_threshold)
    
# cv2.imshow('threshold image', threshold_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

custom_config = r'--psm 11'
text = pytesseract.image_to_string(threshold_img, config=custom_config, output_type=Output.DICT)
print(text)

results = pytesseract.image_to_data(gray_image, config=custom_config, output_type=Output.DICT)
print(results)

for i in range(0, len(results["text"])):
    # extract the bounding box coordinates of the text region from
    # the current result
    x = results["left"][i]
    y = results["top"][i]
    w = results["width"][i]
    h = results["height"][i]
    # extract the OCR text itself along with the confidence of the
    # text localization
    text = results["text"][i]
    conf = int(float(results["conf"][i]))
    
    if (conf > 20) and (len(text) > 3):
        # display the confidence and text to our terminal
        print("Confidence: {}".format(conf))
        print("Text: {}".format(text))
        print("")
        # strip out non-ASCII text so we can draw the text on the image
        # using OpenCV, then draw a bounding box around the text along
        # with the text itself
        text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        
# show the output image
# cv2.imshow("Image", img)
# cv2.waitKey(0)

cv2.imwrite('C:/Users/elean/PythonProj/02_Medias/AFTRER.jpg', img)

103.0
{'text': 'oy,\n\n‘\n\nes\n\n8\n\nc\n\n"ham\n\nE20\n\nre\n\n*,\n\nen\n\nse\n\n5 Re a\n\nm= == me\n\nXG801\n\nGPON-8\n\n=\n\x0c'}
{'level': [1, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5], 'page_num': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'block_num': [0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 16], 'par_num': [0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,

True